In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt


# Load and preprocess the data
data = pd.read_csv('df1.csv')  # Replace 'your_data.csv' with the path to your dataset
data = data[['year-week', 'ID_material', 'Units']]
data = data.pivot(index='year-week', columns='ID_material', values='Units')
data = data.fillna(0)  # Replace NaN values with 0 or handle them based on your requirements

# Scale the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Generate input sequences and target values
sequence_length = 10
input_sequences = []
target_values = []
for i in range(len(scaled_data) - sequence_length - 8):  # Adjust -8 to match the desired prediction length
    input_sequences.append(scaled_data[i:i+sequence_length])
    target_values.append(scaled_data[i+sequence_length:i+sequence_length+8])

input_sequences = np.array(input_sequences)
target_values = np.array(target_values)

# Split the data into training and validation sets
split_ratio = 0.8  # 80% for training, 20% for validation
split_index = int(split_ratio * len(input_sequences))

x_train = input_sequences[:split_index]
y_train = target_values[:split_index]
x_val = input_sequences[split_index:]
y_val = target_values[split_index:]

# Build the RNN model
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(sequence_length, data.shape[1])))
model.add(Dense(8))  # 8 is the length of the target values to be predicted

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

# Make predictions on new data
new_input_sequence = scaled_data[-sequence_length:]  # Use the last available sequence as input
new_input_sequence = np.expand_dims(new_input_sequence, axis=0)
predictions = model.predict(new_input_sequence)

# Inverse scale the predictions
predictions = scaler.inverse_transform(predictions)

print("Predictions:")
print(predictions)

actual_values = data.iloc[-8:].values  # Assuming you have the actual values for the last 8 time steps
given_array = actual_values
# Convert shape to (1, 8)
converted_array = given_array.transpose()  # or converted_array = given_array.T

print(converted_array)
print(converted_array.shape)

# Calculate RMSE

rmse = np.sqrt(mean_squared_error(converted_array, predictions))
print("RMSE:", rmse)



Epoch 1/10
7/7 [==============================] - 2s 93ms/step - loss: 0.2083 - val_loss: 0.2413
Epoch 2/10
7/7 [==============================] - 0s 14ms/step - loss: 0.1821 - val_loss: 0.2073
Epoch 3/10
7/7 [==============================] - 0s 14ms/step - loss: 0.1489 - val_loss: 0.1605
Epoch 4/10
7/7 [==============================] - 0s 15ms/step - loss: 0.1069 - val_loss: 0.1187
Epoch 5/10
7/7 [==============================] - 0s 15ms/step - loss: 0.0930 - val_loss: 0.1134
Epoch 6/10
7/7 [==============================] - 0s 15ms/step - loss: 0.0832 - val_loss: 0.1089
Epoch 7/10
7/7 [==============================] - 0s 15ms/step - loss: 0.0825 - val_loss: 0.1079
Epoch 8/10
7/7 [==============================] - 0s 14ms/step - loss: 0.0815 - val_loss: 0.1073
Epoch 9/10
7/7 [==============================] - 0s 14ms/step - loss: 0.0813 - val_loss: 0.1068
Epoch 10/10
1/1 [==============================] - 0s 334ms/step
Predictions:
[[101.59208  104.3542   101.9696   109.09141  114

In [ ]:
predictions.sum()

In [ ]:
converted_array.sum()